In [1]:
from utilities.google_client import google_sheet_API
from utilities.athena_client import athena_API
import multiprocessing as mp

import pandas as pd
from configparser import ConfigParser
import os
import pyathena
import time

In [5]:
string = '/auth/login'
string.replace('/', '%2F')

'%2Fauth%2Flogin'

In [2]:
# Initialise the 2 APIs 
gsheetAPI = google_sheet_API()
athenaAPI = athena_API()

In [3]:
# Retrieve the sheet and add the queries
gsheet_df = gsheetAPI.retrieve_sheet_as_df()
gsheet_df_queries = athenaAPI.add_row_queries(gsheet_df)

# Get all the queries into one array
queries = gsheet_df_queries['tw_query'].append(gsheet_df_queries['lw_query'], ignore_index=True)

In [8]:
# Prepare a list of dicts
queries_list = []
for row in gsheet_df_queries.iterrows():
    queries_list.append({'row_idx' : row[0],
                         'tw_query': row[1]['tw_query'],
                         'lw_query': row[1]['lw_query']})

In [11]:
queries_list[1]

{'row_idx': 1,
 'tw_query': "SELECT COUNT(*)\n                    FROM bon_log_prod.access\n                    WHERE account_id = '20f61841-e844-4532-8c08-dcd517daabb8'\n                      and site_id = 'f44b7a05-3268-451b-ae56-79dddd9199c9'\n                      and access_time > date_add('minute', - 60, NOW())  -- 60*24*7 + 60\n                      and ds >= CAST(DATE(date_add('day', -1, NOW())) as VARCHAR) -- efficency, reduces the search scope\n                      and regexp_like(request_path, 'realestateandhomes-detai') = true\n                      AND action is NULL\n                      ",
 'lw_query': "SELECT COUNT(*)\n                    FROM bon_log_prod.access\n                    WHERE account_id = '20f61841-e844-4532-8c08-dcd517daabb8'\n                      and site_id = 'f44b7a05-3268-451b-ae56-79dddd9199c9'\n                      and access_time > date_add('minute', - 10140, NOW())  -- 60*24*7 + 60\n                      and access_time <= date_add('minute', -

In [ ]:
def get_data(query):
    row_idx = query['row_idx']
    tw_query = athenaAPI.get_pandas_df(query['tw_query'])
    lw_query = athenaAPI.get_pandas_df(query['lw_query'])
    return {'row_idx' : row_idx,
            'tw_query': tw_query,
            'lw_query': lw_query}

In [ ]:
# Multiprocessing to run the queries simultaneously
# It has to stay in the main function
pool = mp.Pool(int(mp.cpu_count()))
results = [pool.apply(athenaAPI.get_pandas_df, args=[query]) for query in queries_list]